In [1]:
import requests
import pandas as pd
import sqlite3
import lxml.html

Variables used: request_web(for website data), html_to_xml,trow(<tr>),tr(table requested), tablebody(table rows), data_sample(sample for loop), data(whole table data), df(panda dataframe)

In [2]:
request_web = requests.get("https://www.worldwildlife.org/species/directory#")

In [3]:
request_web

<Response [200]>

In [4]:
html_to_xml= lxml.html.fromstring(request_web.text)

In [5]:
html_to_xml

<Element html at 0x7ff3d18aee08>

In [6]:
trow = html_to_xml.xpath('//tr') # the website has one table, which has <thead> and <tbody> both having <tr>, 
#this code will extract the all the rows of table at once

In [7]:
[len(T) for T in trow[:10]]#cross verifying to check if all the rows are of same length, if yes they are from same table

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [8]:
tr = html_to_xml.xpath('//tbody') # it better to pull the table using <tbody> tag, instead of <tr> tag

In [9]:
[len(T) for T in tr[:10]]

[50]

In [10]:
len(tr) #the one table in website

1

In [11]:
tr

[<Element tbody at 0x7ff3d10215e8>]

In [12]:
len(tr[0].getchildren())

50

In [13]:
tablebody=tr[0]
len(tablebody)

50

In [14]:
tablebody.getchildren()[0].text_content()

'\nAfrican Elephant\nLoxodonta africana\nVulnerable\n'

In [15]:
tablebody.getchildren()[0].getchildren()

[<Element td at 0x7ff3d1021e08>,
 <Element td at 0x7ff3d1021ae8>,
 <Element td at 0x7ff3d1021cc8>]

In [16]:
print(tablebody.getchildren()[0].getchildren()[0].text_content())
print(tablebody.getchildren()[0].getchildren()[1].text_content())
print(tablebody.getchildren()[0].getchildren()[2].text_content())

African Elephant
Loxodonta africana
Vulnerable


In [17]:
tablebody.getchildren()[0]

<Element tr at 0x7ff3d101e548>

In [18]:
tablebody.getchildren()[0].getchildren()[0].text_content()

'African Elephant'

In [19]:
tablebody.getchildren()[0].getchildren()[1].text_content()

'Loxodonta africana'

In [20]:
tablebody.getchildren()[0].getchildren()[2].text_content()

'Vulnerable'

checking for small sample of data

In [21]:
data_sample = {
    "Common name"         :[],
    "Scientific name"     :[],
    "Conservation status" :[]  
}
for row in tablebody.getchildren()[0:3]:
    data_sample["Common name"].append(row.getchildren()[0].text_content())
    data_sample["Scientific name"].append(row.getchildren()[1].text_content())
    data_sample["Conservation status"].append(row.getchildren()[2].text_content())

In [22]:
data_sample

{'Common name': ['African Elephant', 'African Wild Dog', 'Albacore Tuna'],
 'Scientific name': ['Loxodonta africana',
  'Lycaon pictus',
  'Thunnus alalunga'],
 'Conservation status': ['Vulnerable', 'Endangered', 'Near Threatened']}

In [23]:
pd.DataFrame(data_sample)

,Common name,Scientific name,Conservation status
0,African Elephant,Loxodonta africana,Vulnerable
1,African Wild Dog,Lycaon pictus,Endangered
2,Albacore Tuna,Thunnus alalunga,Near Threatened


In [24]:
data = {
    "Common name"         :[],
    "Scientific name"     :[],
    "Conservation status" :[]  
}
for row in tablebody.getchildren():
    data["Common name"].append(row.getchildren()[0].text_content().strip())
    data["Scientific name"].append(row.getchildren()[1].text_content().strip())
    data["Conservation status"].append(row.getchildren()[2].text_content().strip())   
       
pd.DataFrame(data).head()

,Common name,Scientific name,Conservation status
0,African Elephant,Loxodonta africana,Vulnerable
1,African Wild Dog,Lycaon pictus,Endangered
2,Albacore Tuna,Thunnus alalunga,Near Threatened
3,Amazon River Dolphin,Scientific Name Inia geoffrensis,
4,Amur Leopard,Panthera pardus orientalis,Critically Endangered


In [25]:
df=pd.DataFrame(data)   

In [26]:
import numpy as np

In [27]:
df.replace('', np.nan, inplace=True)

In [28]:
for i in df.itertuples():
    print(i[1:])

('African Elephant', 'Loxodonta africana', 'Vulnerable')
('African Wild Dog', 'Lycaon pictus', 'Endangered')
('Albacore Tuna', 'Thunnus alalunga', 'Near Threatened')
('Amazon River Dolphin', 'Scientific Name Inia geoffrensis', nan)
('Amur Leopard', 'Panthera pardus orientalis', 'Critically Endangered')
('Amur Tiger', 'Panthera tigris altaica', 'Endangered')
('Arctic Fox', 'Vulpes lagopus', 'Least Concern')
('Arctic Wolf', 'Canis lupus arctos', 'Least Concern')
('Asian Elephant', 'Elephas maximus indicus', 'Endangered')
('Beluga', 'Delphinapterus leucas', 'Near Threatened')
('Bengal Tiger', 'Panthera tigris tigris', 'Endangered')
('Bigeye Tuna', 'Thunnus obesus', 'Vulnerable')
('Black Rhino', 'Diceros bicornis', 'Critically Endangered')
('Black Spider Monkey', 'Ateles paniscus', 'Vulnerable')
('Black-footed Ferret', 'Mustela nigripes', 'Endangered')
('Blue Whale', 'Balaenoptera musculus', 'Endangered')
('Bluefin Tuna', 'Thunnus spp', 'Endangered')
('Bonobo', 'Pan paniscus', 'Endangered'

To use sqlite3 module, you must first create a connection object that represents the database and then optionally you can create a cursor object, which will help you in executing all the SQL statements.

In [29]:
conn = sqlite3.connect(":memory:") # database(stored in variable conn) is created temprorarily in memory
cursor = conn.cursor()
cursor.execute("""
    CREATE TABLE SPECIES(Common_name PRIMARY KEY NOT NULL, Scientific_name, Conservation_status)
""")
for ro in df.itertuples():
    insert_table = """
    INSERT INTO SPECIES(Common_name, Scientific_name, Conservation_status) \
    VALUES (?,?,?)
"""
    cursor.execute(insert_table, ro[1:5])
#conn.close()
conn.commit()

NOT NULL is the constraints for not allowing NULL in the records of column Common_name.A primary key column cannot have NULL values.primary keyword in a relational database is unique for each record/row. CREATE TABLE- is a statement, CREATE-Data Definition Language and it is a command
creating Records in SPECIES table using INSERT-Data Manipulation Language ,INSERT INTO -is a statement and INSERT is a command. Note: never give a space after '\', or else will raise error

In [30]:
data_None = cursor.execute("""
                      SELECT * FROM SPECIES 
""")
for row in data_None:
   print("Common_name = ", row[0])
   print("Scientific_name = ", row[1])
   print("Conservation_status = ", row[2],"\n")
conn.commit()

Common_name =  African Elephant
Scientific_name =  Loxodonta africana
Conservation_status =  Vulnerable 

Common_name =  African Wild Dog
Scientific_name =  Lycaon pictus
Conservation_status =  Endangered 

Common_name =  Albacore Tuna
Scientific_name =  Thunnus alalunga
Conservation_status =  Near Threatened 

Common_name =  Amazon River Dolphin
Scientific_name =  Scientific Name Inia geoffrensis
Conservation_status =  None 

Common_name =  Amur Leopard
Scientific_name =  Panthera pardus orientalis
Conservation_status =  Critically Endangered 

Common_name =  Amur Tiger
Scientific_name =  Panthera tigris altaica
Conservation_status =  Endangered 

Common_name =  Arctic Fox
Scientific_name =  Vulpes lagopus
Conservation_status =  Least Concern 

Common_name =  Arctic Wolf
Scientific_name =  Canis lupus arctos
Conservation_status =  Least Concern 

Common_name =  Asian Elephant
Scientific_name =  Elephas maximus indicus
Conservation_status =  Endangered 

Common_name =  Beluga
Scientifi

Retreiving speccific data

In [31]:
data_None = cursor.execute("""
                      SELECT * FROM SPECIES
                      WHERE Conservation_status="Least Concern"
""")
for row in data_None:
   print("Common_name = ", row[0])
   print("Scientific_name = ", row[1])
   print("Conservation_status = ", row[2],"\n")
conn.commit()

Common_name =  Arctic Fox
Scientific_name =  Vulpes lagopus
Conservation_status =  Least Concern 

Common_name =  Arctic Wolf
Scientific_name =  Canis lupus arctos
Conservation_status =  Least Concern 

Common_name =  Bowhead Whale
Scientific_name =  Balaena mysticetus
Conservation_status =  Least Concern 

Common_name =  Brown Bear
Scientific_name =  Ursus arctos
Conservation_status =  Least Concern 

Common_name =  Common Bottlenose Dolphin
Scientific_name =  Tursiops truncates
Conservation_status =  Least Concern 

Common_name =  Gray Whale
Scientific_name =  Eschrichtius robustus
Conservation_status =  Least Concern 



Updating a values of table in database using Primary Key as reference 

In [32]:
cursor.execute("""UPDATE SPECIES SET Conservation_status="Least-Concern" WHERE Common_name="Gray Whale"
""")
conn.commit()

In [33]:
data_None = cursor.execute("""
                      SELECT * FROM SPECIES
                      WHERE Conservation_status="Least-Concern" 
""")
for row in data_None:
   print("Common_name = ", row[0])
   print("Scientific_name = ", row[1])
   print("Conservation_status = ", row[2],"\n")
conn.commit()

Common_name =  Gray Whale
Scientific_name =  Eschrichtius robustus
Conservation_status =  Least-Concern 



Inserting a data in the table in database

In [34]:
cursor.execute("INSERT INTO SPECIES(Common_name, Scientific_name, Conservation_status) \
      VALUES ('Cat', 'Felis catus', 'Important' )")
conn.commit()

In [35]:
data_None = cursor.execute("""
                      SELECT * FROM SPECIES
                      WHERE Conservation_status="Important"
""")
for row in data_None:
   print("Common_name = ", row[0])
   print("Scientific_name = ", row[1])
   print("Conservation_status = ", row[2],"\n")
conn.commit()

Common_name =  Cat
Scientific_name =  Felis catus
Conservation_status =  Important 



Deleting the same data as inserted above

In [36]:
cursor.execute("""DELETE from SPECIES where Common_name="Cat"
""")
conn.commit()

In [37]:
data_None = cursor.execute("""
                      SELECT * FROM SPECIES
                      WHERE Conservation_status="Important"
""")
for row in data_None:
   print("Common_name = ", row[0])
   print("Scientific_name = ", row[1])
   print("Conservation_status = ", row[2],"\n")
conn.commit()

The data base does not return any values, since it does not have values we ased for.

In [38]:
conn.close()

https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059- Xpath example